In [1]:
import torch
import threading
from diffusers import AutoPipelineForImage2Image
from diffusers import DDIMScheduler
from diffusers_local import UNet2DConditionModel as UNet2DConditionModel_local
from diffusers.utils import load_image
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from NetworkBending import NetworkBending
import time  # for safely terminating the loop

# Create a global lock for synchronizing OSC parameter updates
osc_lock = threading.Lock()

# Instantiate NetworkBending object
NB = NetworkBending()

# Start the OSC server in a background thread
osc_thread = threading.Thread(target=NB.serve_forever)
osc_thread.daemon = True  # Ensure the OSC thread terminates when the program ends
osc_thread.start()

# Initialize the pipeline
pipe = AutoPipelineForImage2Image.from_pretrained("teticio/audio-diffusion-256", torch_dtype=torch.float32, variant="fp16", )
pipe.unet = UNet2DConditionModel_local.from_pretrained("teticio/audio-diffusion-256", subfolder="unet", torch_dtype=torch.float32, variant="fp16", network_bending=NB)
#pipe.load_lora_weights("Danhearn/spectrogram", weight_name="pytorch_lora_weights.safetensors", adapter_name="amen")
pipe.to("cuda")

init_image = load_image(r"C:\\Users\\danhearn\Documents\\GitHub\\diffusion-models-class\\unit4\\spectrograms\\cw_amen17_175.png")

# pipe.scheduler = DDIMScheduler(
#     beta_start=0.00085,
#     beta_end=0.012,
#     beta_schedule="scaled_linear",
#     clip_sample=False,
#     set_alpha_to_one=False,
#     steps_offset=1,
#     prediction_type="epsilon",
#     num_train_timesteps=1000,
#     timestep_spacing="trailing",
#     trained_betas=None
# )

#pipe.set_progress_bar_config(disable=True)

#print(pipe.scheduler)
#print(pipe.unet)

# Main loop for image generation
try:
    while True:
        with osc_lock:  # Ensure that network bending is thread-safe
            output = pipe(prompt=" ", image=init_image, generator=torch.Generator(device="cpu").manual_seed(42), strength=0.5, guidance_scale=0, num_inference_steps=4, width=256, height=256)
        
        image = output.images[0]
        clear_output(wait=True)
        plt.imshow(image)
        plt.axis('off')  # Hide axes
        display(plt.gcf())
        #time.sleep(0.1)  # Add a small sleep to prevent overwhelming CPU

except KeyboardInterrupt:
    NB.stop()
    print("Terminating the loop gracefully.")
    


c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\kornia\feature\lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


OSC server running...


ValueError: AutoPipeline can't find a pipeline linked to AudioDiffusionPipeline for None